#### Pipeline

In [1]:
# Data manipulation
import numpy as np
import pandas as pd
# Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
# Checking whether a numerical feature has a normal distribution or not
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import cross_validate, learning_curve, train_test_split, StratifiedKFold, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, classification_report, precision_recall_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from scipy.stats import randint

In [2]:
url = '/home/fernando_cir/code/monrosegregory/credit_score/raw_data/train.csv'
df = pd.read_csv(url)
df.head(4)

/tmp/ipykernel_87052/471918335.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good


In [3]:
data = df.copy()
data.head(2)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.82262,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.94496,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good


In [4]:
data.shape

(100000, 28)

In [5]:
data['Payment_Behaviour'].value_counts(ascending=False)

Payment_Behaviour
Low_spent_Small_value_payments      25513
High_spent_Medium_value_payments    17540
Low_spent_Medium_value_payments     13861
High_spent_Large_value_payments     13721
High_spent_Small_value_payments     11340
Low_spent_Large_value_payments      10425
!@9#%8                               7600
Name: count, dtype: int64

In [6]:
# DEF DATA CLEANING: dropping  useless features, duplicates and cleaning special characters with nan

def cleaning(df: pd.DataFrame):
    #dropping useless features:
    df = df.drop(['ID', 'Name', 'SSN', 'Monthly_Inhand_Salary', 'Type_of_Loan'], axis = 1)
    #dropping duplicates
    df = df.drop_duplicates()
    #removing _ for empty
    df = df.replace('_', '', regex=True)
    #removing empty for nan
    df.replace('', np.nan, inplace=True)
    #Identify rows with empty values
    rows_with_empty_values = df[df.eq('').any(axis=1)]
    #Fill empty values with NaN in selected rows
    df.loc[rows_with_empty_values.index] = df.loc[rows_with_empty_values.index].replace('', np.nan)
    #cleaning special characters
    df.replace('!@9#%8', np.nan, inplace=True)
    #Communicating
    print("Data Cleaning DONE ✅")
    #Returning df
    return df

In [7]:
#Applying the function
data = cleaning(data)

Data Cleaning DONE ✅


In [8]:
#Simple sanity check
data['Payment_Behaviour'].value_counts(ascending=False)

Payment_Behaviour
LowspentSmallvaluepayments      25513
HighspentMediumvaluepayments    17540
LowspentMediumvaluepayments     13861
HighspentLargevaluepayments     13721
HighspentSmallvaluepayments     11340
LowspentLargevaluepayments      10425
Name: count, dtype: int64

In [11]:
#Handling date and cronological features (enconding AND hadling with missing values)

def cronological(df: pd.DataFrame):
    months_in_year = 12
    dic_date = {"January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6,
                "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12}

    df['Month'] = df['Month'].map(dic_date)

    df["sin_Month"] = np.sin(2*np.pi*df[["Month"]] / months_in_year)
    df["cos_Month"] = np.cos(2*np.pi*df[["Month"]] / months_in_year)

    # Extract years and months from 'Credit_History_Age' column
    df['Credit_History_Years'] = df['Credit_History_Age'].str.extract(r'(\d+)\s*Years?').astype(float)
    df['Credit_History_Months'] = df['Credit_History_Age'].str.extract(r'(\d+)\s*Months?').astype(float)

    # Handle missing or NaN values
    df['Credit_History_Years'] = df['Credit_History_Years'].fillna(0)
    df['Credit_History_Months'] = df['Credit_History_Months'].fillna(0)

    # Convert years to months and add with months to get total months
    df['Credit_History_Age_Months'] = (df['Credit_History_Years'] * 12 + df['Credit_History_Months']).astype(int)

    # Drop the intermediate columns used for calculation (optional, if you no longer need them)
    df.drop(columns=['Credit_History_Years', 'Credit_History_Months'], inplace=True)

    # Display counts of unique values in 'Credit_History_Age_Months'
    value_counts = df['Credit_History_Age_Months'].value_counts()

    # Replace 0 with NaN in the 'Credit_History_Age_Months' column
    df['Credit_History_Age_Months'] = df['Credit_History_Age_Months'].replace(0, np.nan)

    # Group by 'Customer_ID' and calculate the mean for 'Credit_History_Age_Months' within each group
    mean_by_Customer_ID = df.groupby('Customer_ID')['Credit_History_Age_Months'].transform(lambda x: x.mean())

    # Fill NaN values in 'Credit_History_Age' with the mean for each group
    df['Credit_History_Age_Months'] = df['Credit_History_Age_Months'].fillna(mean_by_Customer_ID)

    df.drop(columns=['Credit_History_Age', 'Month'], inplace=True)

    print("Cronological features: DONE ✅")
    return df

In [ ]:
#Handling with missing values. colar todos os tratamentos e ver se ha um padrão e simplificar nesse padrão

In [10]:
#Pulei history age

def missing(df: pd.DataFrame):
    list_to_num = ["Num_of_Delayed_Payment", "Amount_invested_monthly", "Changed_Credit_Limit", "Num_Credit_Inquiries", "Monthly_Balance"]
    list_mode = ["Credit_Mix", "Occupation",]
    list_mean = ["Num_of_Delayed_Payment", "Amount_invested_monthly", "Changed_Credit_Limit", "Num_Credit_Inquiries", "Monthly_Balance"]

    for feature in list_to_num:
        df[feature] = pd.to_numeric(df[feature], errors='coerce')

    for feature in list_mode:
        mode_by_Customer_ID = df.groupby('Customer_ID')[feature].transform(lambda x: x.mode()[0])
        df[feature] = df[feature].fillna(mode_by_Customer_ID)

    for feature in list_mean:
        mean_by_Customer_ID = df.groupby('Customer_ID')[feature].transform(lambda x: x.mean())
        df[feature] = df[feature].fillna(mean_by_Customer_ID)

    print("Data Missing DONE ✅")
    return df

In [ ]:
#Pulei history age

def missing(df: pd.DataFrame):
    list_to_num = ["Num_of_Delayed_Payment", "Amount_invested_monthly", "Changed_Credit_Limit", "Num_Credit_Inquiries", "Monthly_Balance"]
    list_mode = ["Credit_Mix", "Occupation",  ]
    list_mean = ["Num_of_Delayed_Payment", "Amount_invested_monthly", "Changed_Credit_Limit", "Num_Credit_Inquiries", "Monthly_Balance"]


    #Credit Mix
    mode_by_Customer_ID = df.groupby('Customer_ID')['Credit_Mix'].transform(lambda x: x.mode()[0])
    df['Credit_Mix'] = df['Credit_Mix'].fillna(mode_by_Customer_ID)

    #Credit history age #PULEI, VOLTAR
    #PULEI VOLTAR

    #Occupation
    mode_by_Customer_ID = df.groupby('Customer_ID')['Occupation'].transform(lambda x: x.mode()[0])
    df['Occupation'] = df['Occupation'].fillna(mode_by_Customer_ID)

    #num_of_delay_payments
    df['Num_of_Delayed_Payment'] = pd.to_numeric(df['Num_of_Delayed_Payment'], errors='coerce')
    mean_by_Customer_ID = df.groupby('Customer_ID')['Num_of_Delayed_Payment'].transform(lambda x: x.mean())
    df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].fillna(mean_by_Customer_ID)

    #Amount_invested_monthly
    df['Amount_invested_monthly'] = pd.to_numeric(df['Amount_invested_monthly'], errors='coerce')
    mean_by_Customer_ID = df.groupby('Customer_ID')['Amount_invested_monthly'].transform(lambda x: x.mean())
    df['Amount_invested_monthly'] = df['Amount_invested_monthly'].fillna(mean_by_Customer_ID)

    #Changed_Credit_Limit
    df['Changed_Credit_Limit'] = pd.to_numeric(df['Changed_Credit_Limit'], errors='coerce')
    mean_by_Customer_ID = df.groupby('Customer_ID')['Changed_Credit_Limit'].transform(lambda x: x.mean())
    df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].fillna(mean_by_Customer_ID)

    #Num_Credit_Inquiries
    df['Num_Credit_Inquiries'] = pd.to_numeric(df['Num_Credit_Inquiries'], errors='coerce')
    mean_by_Customer_ID = df.groupby('Customer_ID')['Num_Credit_Inquiries'].transform(lambda x: x.mean())
    df['Num_Credit_Inquiries'] = df['Num_Credit_Inquiries'].fillna(mean_by_Customer_ID)

    #Monthly_Balance
    df['Monthly_Balance'] = pd.to_numeric(df['Monthly_Balance'], errors='coerce')
    mean_by_Customer_ID = df.groupby('Customer_ID')['Monthly_Balance'].transform(lambda x: x.mean())
    df['Monthly_Balance'] = df['Monthly_Balance'].fillna(mean_by_Customer_ID)


    print("Data Missing DONE ✅")
    return df


In [9]:
# Handling with outliers

In [ ]:
# handling with specific columns (all together)

In [ ]:
# Scaling

In [ ]:
#Testar se as features de teste se estão iguais as features de train